In [2]:
import pandas as pd 
import numpy as np
from datetime import date, datetime, timedelta
import lxml.html
import json
import requests 
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json

In [260]:
def scrapping(url):
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    session=requests.Session()
    session.verify=False
    r = session.get( url.replace(' ',''), headers={
    'Upgrade-Insecure-Requests': '1',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded', \
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
    session.close()
    import json
    a=json.loads(r.text)
    a=a['data']
    Npages=a['total_pages']
    next_url=a['next']
    nspot=a['page_size']
    spots=a['results']
    return(Npages,next_url,nspot,spots,r)
def get_data(spots,estado_name):
    data_row=[]
    from pandas import json_normalize
    for i in spots:
        curr_spot=i
        data_row=data_row+[json_normalize(curr_spot).values[0].tolist()+[estado_name]]
    return data_row

def scrapping_paso2(url2):
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    session=requests.Session()
    session.verify=False
    r = session.get( url2.replace(' ',''), headers={
    'Upgrade-Insecure-Requests': '1',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded', \
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
    session.close()
    import json
    a=json.loads(r.text)
    data_row=[]
    from pandas import json_normalize
    for i in a:
       curr_spot=i
       data_row=data_row+[json_normalize(curr_spot).values[0].tolist()]
    return(data_row)

In [167]:
Lista_estados=[
["Aguascalientes",1],
["Baja California",2],
["Baja California Sur",3],
["Campeche",4],
["Chiapas",7],
["Chiuahua",8],
["Cohauila",5],
["Colima",6],
["Ciudad de mexico",9],
["Durango",10],
["Guanajuato",11],
["Guerrero",12],
["Hidalgo",13],
["Jalisco",14],
["México",15],
["Michoacán",16],
["Morelos",17],
["Nayarit",18],
["Nuevo León",19],
["Oaxaca",20],
["Puebla",21],
["Querétaro",22],
["Quintana Roo",23],
["San Luis Potosí",24],
["Sinaloa",25],
["Sonora",26],
["Tabasco",27],
["Tamaulipas",28],
["Tlaxcala",29],
["Veracruz",30],
["Yucatán",31],
["Zacatecas",32]]
Lista_estados=pd.DataFrame(Lista_estados,columns=['estado','n'])

In [285]:
espacios=[]
locaciones=[]
for Estado in Lista_estados.n.values.tolist():
    condicion='rent' # rent/sale (Extraer todos )
    type='industrial' # office/industrial (extraer todos)
    tmin='' #en metros cuadrados
    tmax='' #en metros cuadrados
    vmin='' #en usd por metros cuadrados
    vmax='' #en usd por metros cuadrados
    estado_name=Lista_estados.iloc[np.where(Lista_estados.n==Estado)].estado.tolist()[0]
    if ((type=='industrial') * (condicion=='sale')) + ((type=='industrial') * (condicion=='rent')):
        columnas=['id', 'solili_id', 'thumbnail', 'leasing_company', 'building_id',
            'building_name', 'space_slug', 'building_municipality_address',
            'divisible_minimum', 'available_sqm', 'rent_price', 'rent_currency',
            'sale_price', 'sale_currency', 'industrial_park', 'market',
            'all_confirmed', 'is_favorite', 'space_membership', 'is_for_rent','is_for_sale','Estado']
    else:
        columnas=['id', 'solili_id', 'thumbnail', 'leasing_company', 'building_id',
            'building_name', 'space_slug', 'building_municipality_address',
            'divisible_minimum', 'available_sqm', 'rent_price', 'rent_currency',
            'sale_price', 'sale_currency', 'industrial_park', 'market',
            'all_confirmed', 'is_favorite', 'space_membership', 'is_for_rent','Estado']
            
    columnas2=['id', 'lat', 'lng', 'thumbnail', 'available_sqm', 'num_spaces_value',
        'min_rent_price_space_value', 'min_sale_price_space_value', 'label',
        'building_slug', 'url_path', 'url_ws']  

    curr_url="https://www.solili.mx/ws/search/cards" 
    adicional="/@@type/?transaction=@@condicion&search_type=space&building_type=@@type&state=@@estado& \
    boundary=1&size_by=sqm&min_size=@@tmin&max_size=@@tmax& \
    currency=usd&price_by=usd-sqm&min_price=@@vmin&max_price=@@vmax& \
    building_class=A%2CB%2CC&fit_out=all&building_status=all& \
    min_year_built=&max_year_built=&min_floors=&max_floors=& \
    min_typical_floor_size=&max_typical_floor_size=&leasing_company=& \
    min_nra=&max_nra=&building_developer=&industrial_park=&bedrooms=0& \
    baths=0&min_parking_spaces=&max_parking_spaces=&shape_data=&order_by=&verified=false"
    adicional=adicional.replace('@@estado',str(Estado)).replace('@@condicion',condicion) \
        .replace('@@type',type).replace('@@tmin',tmin).replace('@@tmax',tmax).replace('@@vmin',vmin).replace('@@vmax',vmax).replace(' ','')
    curr_url=curr_url+adicional
    Npages,ncurr_url,nspot,spots,r=scrapping(curr_url)
    data_full=[]
    data_full2=[]
    if len(spots)>0:
        spot_extendido=scrapping_paso2(curr_url.replace('https://www.solili.mx/ws/search/cards','https://www.solili.mx/ws/search/points'))
        data_full=data_full+get_data(spots,estado_name)
        data_full2=data_full2+spot_extendido
        for i in range(1,Npages):
            spot_extendido=scrapping_paso2(ncurr_url.replace('http://www.solili.mx/ws/search/cards','https://www.solili.mx/ws/search/points'))
            Npages,ncurr_url,nspot,spots,r=scrapping(ncurr_url)
            data_full=data_full+get_data(spots,estado_name)
            data_full2=data_full2+spot_extendido
    espacios=espacios+data_full
    locaciones=locaciones+data_full2
    print(estado_name+' ok!' +' '+ str(len(data_full)) + ' spot encontrados')
    #Locales=pd.DataFrame(data_full,columns=columnas)
    #Ubicaciones=pd.DataFrame(data_full2,columns=columnas2)
   # Locales['Estado']=Lista_estados.iloc[np.where(Lista_estados.n==Estado)].estado.tolist()[0]
  #  Locales['condicion']=condicion # rent/sale (Extraer todos )
  #  Locales['type']=type # office/industrial (extraer todos)

Aguascalientes ok! 5 spot encontrados
Baja California ok! 43 spot encontrados
Baja California Sur ok! 0 spot encontrados
Campeche ok! 0 spot encontrados
Chiapas ok! 0 spot encontrados
Chiuahua ok! 35 spot encontrados
Cohauila ok! 43 spot encontrados
Colima ok! 0 spot encontrados
Ciudad de mexico ok! 8 spot encontrados
Durango ok! 0 spot encontrados
Guanajuato ok! 59 spot encontrados
Guerrero ok! 0 spot encontrados
Hidalgo ok! 0 spot encontrados
Jalisco ok! 32 spot encontrados
México ok! 91 spot encontrados
Michoacán ok! 0 spot encontrados
Morelos ok! 0 spot encontrados
Nayarit ok! 0 spot encontrados
Nuevo León ok! 97 spot encontrados
Oaxaca ok! 0 spot encontrados
Puebla ok! 6 spot encontrados
Querétaro ok! 53 spot encontrados
Quintana Roo ok! 0 spot encontrados
San Luis Potosí ok! 31 spot encontrados
Sinaloa ok! 0 spot encontrados
Sonora ok! 2 spot encontrados
Tabasco ok! 0 spot encontrados
Tamaulipas ok! 15 spot encontrados
Tlaxcala ok! 0 spot encontrados
Veracruz ok! 0 spot encontrad

In [316]:
Locales=pd.DataFrame(espacios,columns=columnas)
Ubicaciones=pd.DataFrame(locaciones,columns=columnas2) 
Locales.to_csv('locales_'+type+'_'+condicion+'.csv')
Ubicaciones.to_csv('Ubicaciones_'+type+'_'+condicion+'.csv')


GENERAR LOS DATOS ADICIONALES

In [394]:
data=pd.read_csv('locales_industrial_rent.csv')
#url='https://www.solili.mx/ws/search/space-detail/office/'
url='https://www.solili.mx/ws/search/space-detail/industrial/'
spot=(url+data.id.astype('str')).values
datosadicionales=[]
for i in spot:
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    session=requests.Session()
    session.verify=False
    r = session.get(i.replace(' ',''), headers={
    'Upgrade-Insecure-Requests': '1',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded', \
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
    session.close()
    import json
    a=json.loads(r.text)
    datosadicionales=datosadicionales+[json_normalize(a).values[0].tolist()]

KeyboardInterrupt: 

In [380]:
pd.DataFrame(datosadicionales).to_csv('respaldo_otros_industrial_rent.csv')

In [401]:
data=pd.read_csv('otros_industrial_rent.csv')
data.drop('Unnamed: 0',axis=1, inplace=True)
#data.columns=['id', 'divisible_minimum', 'available_sqm', 'is_for_rent',
#       'is_for_sale', 'rent_price', 'currency_rent_price', 'sale_price',
#       'currency_sale_price', 'floor_name', 'fit_out', 'images_gallery',
#       'images_gallery_thumbnail', 'map_image', 'description', 'notes',
#       'contacts', 'date_available', 'updated', 'parking_spaces_included',
#       'cam', 'currency_cam', 'contiguous_space', 'confirmed', 'all_confirmed',
#       'valid', 'is_favorite', 'coworking', 'building.id',
#       'building.position.lat', 'building.position.lng',
#       'building.building_name', 'building.full_address',
#       'building.images_gallery_thumbnail', 'building.market',
#       'building.building_status', 'building.is_for_rent',
#       'building.is_for_sale', 'building.total_spaces', 'building.slug',
##       'building.confirmed', 'other_spaces.contact',
 #      'other_spaces.spaces_list']
data.columns=['id', 'suite', 'divisible_minimum', 'available_sqm', 'is_for_rent',
       'is_for_sale', 'rent_price', 'currency_rent_price', 'sale_price',
       'currency_sale_price', 'images_gallery', 'images_gallery_thumbnail',
       'map_image', 'description', 'notes', 'contacts', 'date_available',
       'updated', 'cam', 'currency_cam', 'confirmed', 'all_confirmed', 'valid',
       'is_favorite', 'building.id', 'building.position.lat',
       'building.position.lng', 'building.building_name',
       'building.full_address', 'building.industrial_park',
       'building.images_gallery_thumbnail', 'building.market',
       'building.building_status', 'building.is_for_rent',
       'building.is_for_sale', 'building.total_spaces', 'building.slug',
       'building.confirmed', 'other_spaces.contact',
       'other_spaces.spaces_list']
data.to_csv('otros_industrial_rent.csv')

In [405]:
pd.read_csv('otros_office_sale.csv').iloc[0]

Unnamed: 0                                                                           0
id                                                                               14806
divisible_minimum                                                                  NaN
available_sqm                                                                     90.9
is_for_rent                                                                      False
is_for_sale                                                                       True
rent_price                                                                         NaN
currency_rent_price                                                                NaN
sale_price                                                                     2623.76
currency_sale_price                                                                USD
floor_name                                                                           1
fit_out                                    